# Population data
This notebook collates processed census data, along with migration matrices and outputs them in a format that will allow for input into the simulation, as well as make it easy to process results after simulation.

## Migration matrix
VGsim needs a matrix with entries containing probability of an individual moving between locations per unit time, this notebook obtains that by calculating number of people flying between countries through reversing an effective distance matrix that is available in some supplementary materials https://www.science.org/doi/10.1126/science.1245200#supplementary-materials. 

### Load libraries and datasets

In [1]:
import pandas as pd
import numpy as np

# Alpha2 code of where to start the pandemic, look in the effective distance matrix for the available countries.
pandemic_start_alpha2 = 'CN'
# Uniform probability of staying in any country per unit time.
# Magnitude as UK citizens leaving per day (from ONS) in 2022 / UK population in 2022 = 0.0029 so 1 minus this for magnitude, time scale in this simulation is per 7 days
in_country_probability = 0.999

# Effective distance data

effective_distances = pd.read_csv('data_sources/effective.distance.matrix.country.csv', header=0)
effective_distances.drop(effective_distances.columns[0], axis=1, inplace=True)

# Population sizes data

population_sizes = pd.read_csv('data_sources/census_2013.csv', header=0)
alpha2_codes = effective_distances.columns

# Sampling data (obtained from 02_sampling_multipliers.ipynb)

sampling_multipliers = pd.read_csv('data_sources/sampling_multipliers.csv', header=0, index_col=0)

effective_distances.head()

,AE,AF,AG,AI,AL,AM,AO,AR,AS,AT,...,VI,VN,VU,WF,WS,YE,YT,ZA,ZM,ZW
0,0.000000,5.614035,11.531617,15.902166,10.178471,7.769530,7.319591,9.130665,13.846666,6.344803,...,10.779283,7.031367,10.655119,14.960192,11.888486,6.435626,12.316025,5.262377,7.236649,8.955652
1,1.548074,0.000000,13.079690,17.450240,10.173316,9.317603,8.867664,10.678739,15.394739,5.369434,...,12.327357,8.579441,12.203193,16.508266,13.436560,7.983700,13.864099,6.810451,8.784722,10.503726
2,7.181742,12.795778,0.000000,5.366969,11.139288,13.218427,12.275860,8.597538,11.985323,8.737326,...,7.430663,11.554362,14.098395,16.611105,13.239721,13.617368,14.920257,8.765838,12.203931,12.459113
3,8.968363,14.582398,2.224032,0.000000,13.363319,14.356298,14.254211,9.569931,12.957716,10.961358,...,2.213855,12.526755,15.070788,17.583498,14.212114,15.403988,14.410679,10.989870,14.427962,14.683145
4,6.413046,10.472095,11.760670,17.127640,0.000000,10.349453,11.707611,8.216086,15.163964,3.566389,...,12.096582,11.296419,17.068165,18.930315,16.418362,11.336016,12.905280,10.016969,13.455061,13.435707


### Flux matrix
Invert effective distance function to get flux. Off diagonals matter only, since diagonal is zero in effective distance matrix.

In [2]:
flux_matrix = effective_distances.apply(lambda x: np.exp(1 - x))

### Clean flux matrix
Remove rows and columns of countries in flux matrix that aren't in population dataset.

In [3]:
missing_countries_columns = []
missing_countries_rows = []
for index, alpha2 in enumerate(alpha2_codes):
    country_mask = population_sizes['alpha2'] == alpha2
    country_data = population_sizes[country_mask]
    if len(country_data['population']) == 0:
        missing_countries_columns.append(alpha2)
        missing_countries_rows.append(index)
flux_matrix = flux_matrix.drop(missing_countries_columns, axis=1).drop(flux_matrix.index[missing_countries_rows])
flux_matrix.index = flux_matrix.columns

# Arbritralily say that 85% of people in each country stay in that country each unit of time. If migration rates are unrealistically high, this number is too low.
np.fill_diagonal(flux_matrix.values, 0)
ordered_columns = np.array(flux_matrix.columns)
target_start_index = flux_matrix.columns.get_loc(pandemic_start_alpha2)
ordered_columns[[0, target_start_index]] = ordered_columns[[target_start_index, 0]]
ordered_matrix = pd.DataFrame(0., index=ordered_columns, columns=ordered_columns)
for row in ordered_columns:
    for col in ordered_columns:
        ordered_matrix.at[row, col] = flux_matrix.loc[row, col]
ordered_matrix.head()

,CN,AF,AG,AL,AM,AO,AR,AS,AT,AU,...,VE,VG,VI,VN,VU,WS,YE,ZA,ZM,ZW
CN,0.000000,0.000078,0.000018,0.000031,0.000228,0.000298,0.000124,4.180121e-06,0.001003,0.019114,...,0.000110,5.528111e-06,0.000090,0.014072,4.108398e-05,2.115453e-05,0.000033,0.000860,0.000017,0.000022
AF,0.004430,0.000000,0.000006,0.000104,0.000244,0.000383,0.000063,5.603303e-07,0.012658,0.006342,...,0.000015,7.410236e-07,0.000012,0.000511,1.363061e-05,3.970733e-06,0.000927,0.002996,0.000416,0.000075
AG,0.001401,0.000008,0.000000,0.000039,0.000005,0.000013,0.000502,1.694864e-05,0.000436,0.000953,...,0.000446,8.488147e-04,0.001611,0.000026,2.048515e-06,4.834559e-06,0.000003,0.000424,0.000014,0.000011
AL,0.001309,0.000077,0.000021,0.000000,0.000087,0.000022,0.000735,7.057790e-07,0.076812,0.000049,...,0.000340,9.333760e-07,0.000015,0.000034,1.051199e-07,2.013217e-07,0.000032,0.000121,0.000004,0.000004
AM,0.010669,0.000202,0.000003,0.000096,0.000000,0.000037,0.000024,5.841516e-07,0.041402,0.000609,...,0.000015,7.725267e-07,0.000013,0.001946,1.308039e-06,3.810450e-07,0.000089,0.000288,0.000040,0.000007


In [4]:
normalising_matrix = (1-in_country_probability)*np.diag(np.reciprocal(ordered_matrix.sum(axis=0)))
normalising_matrix

array([[0.00023195, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01336507, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00112343, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00016248, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00129997,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00193516]])

In [5]:
normalised_matrix = np.matmul(ordered_matrix.to_numpy(),normalising_matrix)
np.fill_diagonal(normalised_matrix, in_country_probability)
probability_matrix = pd.DataFrame(normalised_matrix, index=ordered_matrix.columns, columns=ordered_matrix.columns)
probability_matrix

,CN,AF,AG,AL,AM,AO,AR,AS,AT,AU,...,VE,VG,VI,VN,VU,WS,YE,ZA,ZM,ZW
CN,9.990000e-01,1.037670e-06,2.062721e-08,3.259777e-07,1.612077e-06,5.737221e-07,7.604802e-08,2.206945e-08,5.798315e-07,3.082932e-06,...,1.496026e-07,4.671041e-08,3.969598e-07,1.710825e-05,1.539751e-07,3.243152e-08,8.368131e-08,1.396734e-07,2.243683e-08,4.179451e-08
AF,1.027648e-06,9.990000e-01,6.373878e-09,1.089307e-06,1.726699e-06,7.375112e-07,3.847512e-08,2.958331e-09,7.320577e-06,1.022837e-06,...,2.005369e-08,6.261364e-09,5.321105e-08,6.210766e-07,5.108496e-08,6.087440e-09,2.361492e-06,4.867980e-07,5.408447e-07,1.443118e-07
AG,3.249764e-07,1.007233e-07,9.990000e-01,4.146048e-07,3.492291e-08,2.441227e-08,3.083429e-07,8.948239e-08,2.522832e-07,1.537201e-07,...,6.065759e-07,7.172158e-06,7.122121e-06,3.170687e-08,7.677453e-09,7.411751e-09,8.443363e-09,6.888659e-08,1.770636e-08,2.042150e-08
AL,3.035599e-07,1.028710e-06,2.383674e-08,9.990000e-01,6.153047e-07,4.309189e-08,4.515404e-07,3.726245e-09,4.442195e-05,7.888169e-09,...,4.618992e-07,7.886668e-09,6.702340e-08,4.103709e-08,3.939696e-10,3.086417e-10,8.265975e-08,1.971404e-08,5.067226e-09,7.690567e-09
AM,2.474720e-06,2.703377e-06,3.277042e-09,1.010938e-06,9.990000e-01,7.077407e-08,1.498807e-08,3.084098e-09,2.394362e-05,9.815492e-08,...,2.090623e-08,6.527553e-09,5.547321e-08,2.365400e-06,4.902285e-09,5.841713e-10,2.266167e-07,4.671478e-08,5.190129e-08,1.384865e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WS,1.206977e-06,2.323453e-07,1.740734e-08,1.145327e-08,1.424129e-08,1.056383e-08,1.223522e-07,9.654201e-04,1.429600e-08,2.552794e-05,...,1.262499e-07,3.941900e-08,3.349951e-07,7.721817e-07,3.289965e-06,9.990000e-01,1.947687e-08,1.016529e-07,1.329473e-08,3.013512e-08
YE,3.783170e-07,1.090824e-05,2.468069e-09,3.792655e-07,6.686060e-07,4.278092e-07,1.489819e-08,1.145514e-09,2.745540e-07,3.960592e-07,...,7.765115e-09,2.424502e-09,2.060418e-08,2.404909e-07,1.978092e-08,2.357156e-09,9.990000e-01,1.884961e-07,2.738209e-07,5.100637e-07
ZA,1.457361e-06,5.186249e-06,4.582014e-08,2.210527e-07,3.178842e-07,4.589774e-05,3.139032e-06,1.235885e-08,5.547812e-07,4.771570e-06,...,8.709417e-08,2.615775e-08,2.222968e-07,2.021639e-07,2.383130e-07,2.839811e-08,4.347491e-07,9.990000e-01,5.776298e-05,1.309311e-04
ZM,3.113594e-07,5.328072e-06,1.083865e-08,8.819979e-08,3.265770e-07,1.143751e-05,3.795145e-07,1.494207e-09,1.110189e-07,5.768911e-07,...,1.052984e-08,3.913278e-09,2.687607e-08,1.174665e-07,2.881246e-08,3.433381e-09,9.013246e-07,5.339769e-05,9.990000e-01,4.330032e-04


In [6]:
filtered_pop = population_sizes[population_sizes['alpha2'].isin(alpha2_codes)]
old_pop = filtered_pop.copy(deep=True)
filtered_pop.set_index('alpha2', inplace=True)
filtered_pop = filtered_pop.reindex(ordered_columns)

### Write matrix in .mg format


In [7]:
base_file = '../parameters/manypop'

In [8]:
with open(base_file + '.mg', 'w') as f:
    f.write('#Migration_format_version 0.0.1\n')

probability_matrix.to_csv(base_file + '.mg', header=False, index=None, mode="a", sep=' ', float_format='%.15f')

### Write population data in .pp format 

In [9]:
sampling_multipliers_complete = np.full(shape=len(filtered_pop['population']),fill_value=0.00,dtype=np.double)
outer_index = 0
for index, row in filtered_pop.iterrows():
    if index in sampling_multipliers.index:
        sampling_multipliers_complete[outer_index] = sampling_multipliers.loc[index,'proportion'].astype(float)*10*100
    outer_index+=1
sampling_multipliers_complete

array([1.10702783e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.72200355e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 8.00933399e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.34246178e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.80482607e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.12950329e-02, 0.00000000e+00, 0.00000000e+00,
      

In [10]:

population_data = pd.DataFrame({'size': filtered_pop['population'],
                                'contactDensity': np.full(shape=len(filtered_pop['population']), fill_value=1.00,
                                                          dtype=np.double),
                                'conDenAfterLD': np.full(shape=len(filtered_pop['population']), fill_value=0.2,
                                                         dtype=np.double),
                                'startLD': np.full(shape=len(filtered_pop['population']), fill_value=0.7,
                                                   dtype=np.double),
                                'endLD': np.full(shape=len(filtered_pop['population']), fill_value=0.01,
                                                 dtype=np.double),
                                'samplingMultiplier': sampling_multipliers_complete,
                                'fullname':filtered_pop['fullname']})
# VGsim seems to require 3 columns to be comma-seperated, while the rest are space seperated, so must merge some columns by commas: 
population_data = population_data.reindex(ordered_columns)
population_data['id'] = range(0,len(filtered_pop['population']))
# Extract ID to Full country name to a separate table for relabelling countries after simulations
population_translation_data = population_data[['id','fullname']]
population_translation_data = population_translation_data.rename(columns={'id':'location'})
population_data['conDenAfterLD startLD endLD'] = population_data['conDenAfterLD'].astype(str) + "," + population_data[
    'startLD'].astype(str) + "," + population_data['endLD'].astype(str)
population_data = population_data[['id', 'size', 'contactDensity', 'conDenAfterLD startLD endLD', 'samplingMultiplier']]

In [11]:
translation_table = population_translation_data.merge(old_pop, left_on='fullname', right_on='fullname', how='outer')
translation_table

,location,fullname,alpha2,population
0,1,Afghanistan,AF,32716210
1,3,Albania,AL,2889104
2,51,Algeria,DZ,38760168
3,7,American Samoa,AS,52217
4,5,Angola,AO,27128337
...,...,...,...,...
204,202,Viet Nam,VN,91235504
205,201,Virgin Islands (U.S.),VI,107882
206,205,"Yemen, Rep.",YE,27753304
207,207,Zambia,ZM,15737793


In [12]:
with open(base_file + '.pp', 'w') as f:
    f.write('#Population_format_version 0.0.1\n')

population_data.to_csv(base_file + '.pp', header=True, index=None, mode="a", sep=' ')
translation_table.to_csv(f'{base_file}_country_ids.csv', index=None, sep=',', mode='w')